In [ ]:
!pip install roboflow


from roboflow import Roboflow
rf = Roboflow(api_key="ixMLwwKzqC9U9K5S7XHp")
project = rf.workspace("national-university-fast").project("stationary-items-dataset")
version = project.version(8)
dataset = version.download("yolov8")


In [ ]:
pip install ultralytics

In [ ]:
import os
import cv2

dataset_dir = "/content/Stationary-Items-Dataset-8/"

output_dir = "/content/Modified-Stationary-Items-Dataset/"

os.makedirs(output_dir, exist_ok=True)

def adjust_contrast(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    return cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)

def preprocess_images(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                input_path = os.path.join(root, filename)
                output_path = os.path.join(output_dir, os.path.relpath(root, input_dir), filename)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                img = cv2.imread(input_path)
                if img is None:
                    print("Error: Unable to read image at", input_path)
                    continue
                img_resized = cv2.resize(img, (640, 640))
                img_contrast_adjusted = adjust_contrast(img_resized)
                cv2.imwrite(output_path, img_contrast_adjusted)
                print("Processed:", output_path)

preprocess_images(os.path.join(dataset_dir, "train"), os.path.join(output_dir, "train"))

preprocess_images(os.path.join(dataset_dir, "test"), os.path.join(output_dir, "test"))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def create_simple_cnn(num_classes):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(640, 640, 1), padding="same"),
      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
      tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"),
      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])
  return model

simple_cnn_model = create_simple_cnn(8)
simple_cnn_model.summary()


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras import layers, Model

def create_simple_cnn(num_classes):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(640, 640, 1), padding="same"),
      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
      tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"),
      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])
  return model

model = create_simple_cnn(8)
model.summary()

train_dir = '/content/Modified-Stationary-Items-Dataset/train'
test_dir = '/content/Modified-Stationary-Items-Dataset/test'

image_size = (640, 640)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./250)
test_datagen = ImageDataGenerator(rescale=1/.250)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    color_mode = "grayscale",
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    color_mode = "grayscale",
    batch_size=batch_size,
    class_mode='categorical'
)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    batch_size=batch_size,
    validation_data=test_generator
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model.save("stationary_items_classifier.h5")
